# Waterbomb shell folder and designer

In [1]:
%matplotlib widget

In [2]:
import sympy as sp
sp.init_printing()
import numpy as np
import matplotlib.pylab as plt
import bmcs_utils.api as bu

In [3]:
from bmcs_shell.folding.wb_cell import WBElem
from bmcs_shell.folding.wb_shell import WBShell

In [4]:
data = dict(alpha=1, a=5, b=9, c=4, n_phi_plus=2, n_x_plus=2)

In [5]:
wb = WBElem(**data)
ws = WBShell(wb_cell=wb,**data) 

In [6]:
bu.InteractiveWindow([ws,wb]).interact()

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…

In [7]:
I_m = ws.I_CDij[-1,:,-1,:]
I_m

array([[18, 17],
       [32, 31]])

In [8]:
_, idx_remap = ws.unique_node_map
J_m = idx_remap[I_m]

In [9]:
ws.X_Ia[J_m.flatten()].T

array([[-12.1612097 ,  -2.16120875,   2.16120875,  12.1612097 ],
       [-16.96426964, -16.96426964, -16.96426964, -16.96426964],
       [  0.84221268,   0.84221268,   0.84221268,   0.84221268]])

# Node map to allow for hierarchical sliding

In [10]:
n_cells, n_ic, n_id, _, x_cell_idx, _, y_cell_idx = ws.cell_map
x_cell_idx, y_cell_idx

(array([0, 2]), array([0, 2]))

In [11]:
x_idx, y_idx = x_cell_idx / 2, y_cell_idx / 2
x_idx, y_idx

(array([0., 1.]), array([0., 1.]))

In [9]:
n_x_, n_y_ = len(x_idx), len(y_idx) 

In [10]:
I_cell_offset = (n_ic + np.arange(n_x_*n_y_).reshape(n_x_, n_y_)) * ws.wb_cell.n_I
I_cell_offset

array([[ 7, 14],
       [21, 28]])

In [11]:
ws.wb_cell.I_boundary[np.newaxis,np.newaxis,:,:]

array([[[[2, 1],
         [6, 5],
         [4, 3]]]])

In [12]:
I_CDij_map = I_cell_offset.T[:,:,np.newaxis, np.newaxis] + ws.wb_cell.I_boundary[np.newaxis,np.newaxis,:,:]

In [13]:
I_CDij_map[-1,:,-1,:]

array([[18, 17],
       [32, 31]])

# Find duplicate nodes and merge them to a single one

In [13]:
X_Ia = np.array([[2,1,0],
                 [3,8,9],
                 [0,0,0],
                 [5,3,9],
                 [1,0,1],
                 [0,0,0],
                 [1,0,1],
                 [0,0,0]],dtype=np.float_)

In [14]:
# reshape the coordinates in array of segments to the shape (n_N, n_D
x_0 = X_Ia

In [ ]:
# construct distance vectors between every pair of nodes
x_x_0 = x_0[:, np.newaxis, :] - x_0[np.newaxis, :, :]

In [ ]:
# calculate the distance between every pair of nodes
dist_0 = np.sqrt(np.einsum('...i,...i', x_x_0, x_x_0))

In [ ]:
# identify those at the same location
zero_dist = dist_0 < ws.node_match_threshold
zero_dist

In [17]:
# get their indices
i_idx, j_idx = np.where(zero_dist)

In [18]:
# take only the upper triangle indices
upper_triangle = i_idx < j_idx

In [19]:
idx_multi, idx_delete = i_idx[upper_triangle], j_idx[upper_triangle]

In [20]:
# construct a boolean array with True at valid and False at deleted
# indices
idx_unique = np.ones((len(x_0),), dtype='bool')
idx_unique[idx_delete] = False

In [21]:
idx_multi, idx_delete

(array([2, 2, 4, 5]), array([5, 7, 6, 7]))

In [22]:
idx_keep = np.ones((len(x_0),), dtype=np.bool_)
idx_keep[idx_delete] = False

In [23]:
idx_delete, idx_multi

(array([5, 7, 6, 7]), array([2, 2, 4, 5]))

In [24]:
ij_map = np.ones_like(dist_0, dtype=np.int_) + len(x_0)
ij_map

array([[9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9]])

In [25]:
i_ = np.arange(len(x_0))
idx_row = i_[idx_keep]

In [26]:
ij_map[idx_keep,idx_keep] = np.arange(len(idx_row))
ij_map

array([[0, 9, 9, 9, 9, 9, 9, 9],
       [9, 1, 9, 9, 9, 9, 9, 9],
       [9, 9, 2, 9, 9, 9, 9, 9],
       [9, 9, 9, 3, 9, 9, 9, 9],
       [9, 9, 9, 9, 4, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9]])

In [27]:
ij_map[i_idx, j_idx] = ij_map[i_idx, i_idx]
ij_map

array([[0, 9, 9, 9, 9, 9, 9, 9],
       [9, 1, 9, 9, 9, 9, 9, 9],
       [9, 9, 2, 9, 9, 2, 9, 2],
       [9, 9, 9, 3, 9, 9, 9, 9],
       [9, 9, 9, 9, 4, 9, 4, 9],
       [9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9]])

In [8]:
idx_remap = np.min(ij_map,axis=0)
idx_remap

NameError: name 'ij_map' is not defined

## Sketch of the applied generation method / concept

Sketch the discretization into the cells the cell midpoints 
cover a cylindric surface as a structured grid. 

Midpoints are arranged at the surface of the derived cylinder
parameters which depend on the folding angle $\alpha$ and $a, b, c$
parameters of the waterbomd cell. Then, a reference cell 
in an intermediate state of folding is broadcasted 
to the prepared grid of midpoints on the cylinders. 

In [36]:
n_phi_plus = 2
delta_phi = 0.3
phi_range = np.arange(-(n_phi_plus-1), n_phi_plus) * delta_phi
r_0 = -10

In [37]:
phi_range

array([-0.3,  0. ,  0.3])

In [38]:
X_phi_range = np.array([np.fabs(r_0) * np.sin(phi_range),
                 np.fabs(r_0) * np.cos(phi_range) + r_0]).T

In [39]:
fix, ax = plt.subplots(1,1)
ax.plot(*X_phi_range.T);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
n_x_plus = 2
delta_x = 0.5
X_x_range = np.arange(-(n_x_plus-1), n_x_plus) * delta_x
X_x_range

array([-0.5,  0. ,  0.5])

In [98]:
X_x_range[:, np.newaxis, np.newaxis]

array([[[-0.5]],

       [[ 0. ]],

       [[ 0.5]]])

In [99]:
X_phi_range[np.newaxis,:,:]

array([[[-2.95520207, -0.44663511],
        [ 0.        ,  0.        ],
        [ 2.95520207, -0.44663511]]])

In [100]:
n_idx_x = len(X_x_range)
n_idx_phi = len(X_phi_range)
idx_x = np.arange(n_idx_x)
idx_phi = np.arange(n_idx_phi)
idx_phi

array([0, 1, 2])

In [101]:
1+1 % 2

In [102]:
idx_x_ic = idx_x[(n_idx_x)%2::2]
idx_x_id = idx_x[(n_idx_x+1)%2::2]
idx_phi_ic = idx_phi[(n_idx_phi)%2::2]
idx_phi_id = idx_phi[(n_idx_phi+1)%2::2]

idx_x_ic, idx_phi_ic, idx_x_id, idx_phi_id

(array([1]), array([1]), array([0, 2]), array([0, 2]))

In [103]:
idx_map_ic = np.meshgrid(idx_x_ic, idx_phi_ic)
idx_map_id = np.meshgrid(idx_x_id, idx_phi_id)

In [104]:
idx_map_ic

[array([[1]]), array([[1]])]

In [105]:
X_C_a = np.c_[ X_x_range[idx_map_ic[0]].reshape(-1,1), X_phi_range[idx_map_ic[1]].reshape(-1,2) ]
X_D_a = np.c_[ X_x_range[idx_map_id[0]].reshape(-1,1), X_phi_range[idx_map_id[1]].reshape(-1,2) ]

In [106]:
X_C_a

array([[0., 0., 0.]])

In [107]:
X_D_a

array([[-0.5       , -2.95520207, -0.44663511],
       [ 0.5       , -2.95520207, -0.44663511],
       [-0.5       ,  2.95520207, -0.44663511],
       [ 0.5       ,  2.95520207, -0.44663511]])

In [108]:
from bmcs_shell.folding.waterbomb_shell import WBShell

In [79]:
WBShell().interact()

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…